# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 12 2022 4:03PM,252900,19,MSP216695,"Methapharm, Inc.",Released
1,Dec 12 2022 4:03PM,252899,16,8882968,Sartorius Bioprocess Solutions,Released
2,Dec 12 2022 4:02PM,252897,16,8878251,Sartorius Bioprocess Solutions,Released
3,Dec 12 2022 3:53PM,252896,10,CTF0011540,"Citieffe, Inc.",Released
4,Dec 12 2022 3:51PM,252895,10,CTF0011539,"Citieffe, Inc.",Released
5,Dec 12 2022 3:48PM,252894,10,SNAP1193480,"Snap Medical Industries, LLC",Released
6,Dec 12 2022 3:48PM,252894,10,SNAP1212221,"Snap Medical Industries, LLC",Released
7,Dec 12 2022 3:48PM,252894,10,SNAP1212222,"Snap Medical Industries, LLC",Released
8,Dec 12 2022 3:47PM,252893,22,635903,"NBTY Global, Inc.",Released
9,Dec 12 2022 3:46PM,252892,22,635895,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,252895,Released,1
43,252896,Released,1
44,252897,Released,1
45,252899,Released,1
46,252900,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
252895,NaN,1.0
252896,NaN,1.0
252897,NaN,1.0
252899,NaN,1.0
252900,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
252778,0.0,1.0
252783,1.0,0.0
252784,1.0,0.0
252785,0.0,8.0
252793,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
252778,0,1
252783,1,0
252784,1,0
252785,0,8
252793,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,252778,0,1
1,252783,1,0
2,252784,1,0
3,252785,0,8
4,252793,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,252778,,1
1,252783,1,
2,252784,1,
3,252785,,8
4,252793,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 12 2022 4:03PM,252900,19,"Methapharm, Inc."
1,Dec 12 2022 4:03PM,252899,16,Sartorius Bioprocess Solutions
2,Dec 12 2022 4:02PM,252897,16,Sartorius Bioprocess Solutions
3,Dec 12 2022 3:53PM,252896,10,"Citieffe, Inc."
4,Dec 12 2022 3:51PM,252895,10,"Citieffe, Inc."
5,Dec 12 2022 3:48PM,252894,10,"Snap Medical Industries, LLC"
8,Dec 12 2022 3:47PM,252893,22,"NBTY Global, Inc."
9,Dec 12 2022 3:46PM,252892,22,"NBTY Global, Inc."
10,Dec 12 2022 3:43PM,252891,10,ISDIN Corporation
15,Dec 12 2022 3:41PM,252890,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 12 2022 4:03PM,252900,19,"Methapharm, Inc.",,1
1,Dec 12 2022 4:03PM,252899,16,Sartorius Bioprocess Solutions,,1
2,Dec 12 2022 4:02PM,252897,16,Sartorius Bioprocess Solutions,,1
3,Dec 12 2022 3:53PM,252896,10,"Citieffe, Inc.",,1
4,Dec 12 2022 3:51PM,252895,10,"Citieffe, Inc.",,1
5,Dec 12 2022 3:48PM,252894,10,"Snap Medical Industries, LLC",,3
6,Dec 12 2022 3:47PM,252893,22,"NBTY Global, Inc.",,1
7,Dec 12 2022 3:46PM,252892,22,"NBTY Global, Inc.",,1
8,Dec 12 2022 3:43PM,252891,10,ISDIN Corporation,,5
9,Dec 12 2022 3:41PM,252890,10,"Methapharm, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 4:03PM,252900,19,"Methapharm, Inc.",1,
1,Dec 12 2022 4:03PM,252899,16,Sartorius Bioprocess Solutions,1,
2,Dec 12 2022 4:02PM,252897,16,Sartorius Bioprocess Solutions,1,
3,Dec 12 2022 3:53PM,252896,10,"Citieffe, Inc.",1,
4,Dec 12 2022 3:51PM,252895,10,"Citieffe, Inc.",1,
5,Dec 12 2022 3:48PM,252894,10,"Snap Medical Industries, LLC",3,
6,Dec 12 2022 3:47PM,252893,22,"NBTY Global, Inc.",1,
7,Dec 12 2022 3:46PM,252892,22,"NBTY Global, Inc.",1,
8,Dec 12 2022 3:43PM,252891,10,ISDIN Corporation,5,
9,Dec 12 2022 3:41PM,252890,10,"Methapharm, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 4:03PM,252900,19,"Methapharm, Inc.",1,
1,Dec 12 2022 4:03PM,252899,16,Sartorius Bioprocess Solutions,1,
2,Dec 12 2022 4:02PM,252897,16,Sartorius Bioprocess Solutions,1,
3,Dec 12 2022 3:53PM,252896,10,"Citieffe, Inc.",1,
4,Dec 12 2022 3:51PM,252895,10,"Citieffe, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 4:03PM,252900,19,"Methapharm, Inc.",1.0,NaN
1,Dec 12 2022 4:03PM,252899,16,Sartorius Bioprocess Solutions,1.0,NaN
2,Dec 12 2022 4:02PM,252897,16,Sartorius Bioprocess Solutions,1.0,NaN
3,Dec 12 2022 3:53PM,252896,10,"Citieffe, Inc.",1.0,NaN
4,Dec 12 2022 3:51PM,252895,10,"Citieffe, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 4:03PM,252900,19,"Methapharm, Inc.",1.0,0.0
1,Dec 12 2022 4:03PM,252899,16,Sartorius Bioprocess Solutions,1.0,0.0
2,Dec 12 2022 4:02PM,252897,16,Sartorius Bioprocess Solutions,1.0,0.0
3,Dec 12 2022 3:53PM,252896,10,"Citieffe, Inc.",1.0,0.0
4,Dec 12 2022 3:51PM,252895,10,"Citieffe, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5815510,189.0,15.0
12,252783,0.0,1.0
16,505796,2.0,0.0
19,2275690,15.0,10.0
20,758526,17.0,8.0
21,1011372,4.0,0.0
22,505785,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5815510,189.0,15.0
1,12,252783,0.0,1.0
2,16,505796,2.0,0.0
3,19,2275690,15.0,10.0
4,20,758526,17.0,8.0
5,21,1011372,4.0,0.0
6,22,505785,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,189.0,15.0
1,12,0.0,1.0
2,16,2.0,0.0
3,19,15.0,10.0
4,20,17.0,8.0
5,21,4.0,0.0
6,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,189.0
1,12,Released,0.0
2,16,Released,2.0
3,19,Released,15.0
4,20,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Executing,15.0,1.0,0.0,10.0,8.0,0.0,0.0
Released,189.0,0.0,2.0,15.0,17.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Executing,15.0,1.0,0.0,10.0,8.0,0.0,0.0
1,Released,189.0,0.0,2.0,15.0,17.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Executing,15.0,1.0,0.0,10.0,8.0,0.0,0.0
1,Released,189.0,0.0,2.0,15.0,17.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()